# Projeto 3: Classificação binária brest cancer com tuning dos parâmetros

## Etapa 1: Importação das bibliotecas

In [ ]:
!pip install skorch

In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn
from skorch import NeuralNetBinaryClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
torch.__version__

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


'1.4.0'

## Etapa 2: Base de dados

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

In [ ]:
previsores = np.array(previsores, dtype = 'float32')
classe = np.array(classe, dtype = 'float32').squeeze(1)

In [ ]:
previsores.shape

(569, 30)

In [ ]:
classe.shape

(569,)

## Etapa 3: Classe para estrutura da rede neural

In [ ]:
class classificador_torch(nn.Module):
  def __init__(self, activation, neurons, initializer):
    super().__init__()
    # 30 -> 16 -> 16 -> 1
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation
    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)
    self.output = nn.Sigmoid()

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    X = self.output(X)
    return X

## Etapa 4: Skorch

In [ ]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                  lr = 0.001,
                                                  optimizer__weight_decay = 0.0001,
                                                  train_split=False)

## Etapa 5: Tuning dos parâmetros

In [ ]:
params = {'batch_size': [10],
          'max_epochs': [100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCELoss], #, torch.nn.HingeEmbeddingLoss],
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [8, 16], 
          'module__initializer': [torch.nn.init.uniform]} # _, torch.nn.init.normal_]}

In [ ]:
params

{'batch_size': [10],
 'criterion': [torch.nn.modules.loss.BCELoss],
 'max_epochs': [100],
 'module__activation': [<function torch.nn.functional.relu>,
  <function torch.nn.functional.tanh>],
 'module__initializer': [<function torch.nn.init._make_deprecate.<locals>.deprecated_init>],
 'module__neurons': [8, 16],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD]}

In [ ]:
grid_search = GridSearchCV(estimator=classificador_sklearn, param_grid=params,
                           scoring = 'accuracy', cv = 2)
grid_search = grid_search.fit(previsores, classe)

  epoch    train_loss     dur
-------  ------------  ------
      1       10.3130  0.0511
      2       10.3130  0.0312
      3       10.3130  0.0272
      4       10.3130  0.0383
      5       10.3130  0.0381


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if sys.path[0] == '':


      6       10.3130  0.0272
      7       10.3130  0.0264
      8       10.3130  0.0292
      9       10.3130  0.0268
     10       10.3130  0.0297
     11       10.3130  0.0262
     12       10.3130  0.0265
     13       10.3130  0.0297
     14       10.3130  0.0256
     15       10.3130  0.0310
     16       10.3130  0.0289
     17       10.3130  0.0290
     18       10.3130  0.0284
     19       10.3130  0.0292
     20       10.3130  0.0290
     21       10.3130  0.0291
     22       10.3130  0.0293
     23       10.3130  0.0295
     24       10.3130  0.0301
     25       10.3130  0.0287
     26       10.3130  0.0292
     27       10.3130  0.0339
     28       10.3130  0.0312
     29       10.3130  0.0294
     30       10.3130  0.0433
     31       10.3130  0.0477
     32       10.3130  0.0375
     33       10.3130  0.0306
     34       10.3130  0.0422
     35       10.3130  0.0368
     36       10.3130  0.0283
     37       10.3130  0.0374
     38       10.3130  0.0291
     39   

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


      7        0.7764  0.0324
      8        0.7506  0.0300
      9        0.7285  0.0300
     10        0.7103  0.0386
     11        0.6959  0.0296
     12        0.6851  0.0436
     13        0.6776  0.0368
     14        0.6727  0.0300
     15        0.6695  0.0303
     16        0.6675  0.0291
     17        0.6663  0.0297
     18        0.6655  0.0303
     19        0.6649  0.0297
     20        0.6646  0.0337
     21        0.6644  0.0324
     22        0.6642  0.0298
     23        0.6641  0.0296
     24        0.6640  0.0424
     25        0.6639  0.0268
     26        0.6638  0.0290
     27        0.6638  0.0511
     28        0.6638  0.0425
     29        0.6637  0.0366
     30        0.6637  0.0298
     31        0.6637  0.0299
     32        0.6637  0.0298
     33        0.6637  0.0364
     34        0.6636  0.0335
     35        0.6636  0.0297
     36        0.6636  0.0303
     37        0.6636  0.0305
     38        0.6636  0.0301
     39        0.6636  0.0303
     40   

In [ ]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [ ]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCELoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu>,
 'module__initializer': <function torch.nn.init._make_deprecate.<locals>.deprecated_init>,
 'module__neurons': 8,
 'optimizer': torch.optim.adam.Adam}

In [ ]:
melhor_precisao

0.7524153694094391